some inspiration taken from [@stevernewman on medium](https://medium.com/@stevernewman/eliminating-outliers-in-python-with-z-scores-dd72ca5d4ead)

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

## load csv with all available info

In [39]:
df = pd.read_csv("data/00-completo-sem-target.csv")
df.head(5)

,week_number,sales_depot_id,sales_channel_id,route_id,client_id,product_id,sales_unit_this_week,sales_this_week,returns_unit_next_week,returns_next_week,town,state,client_name,short_name,brand,weight,pieces,target
0,5,1936,1,1158,1575677,2425,60,270.00,0,0.0,2300 LEON MILENIUM,GUANAJUATO,JUAN CARLOS GOMEZ GOMEZ,Nito,BIM,62.0,1.0,60
1,6,1310,1,2005,32695,43066,1,9.63,0,0.0,2260 GONZALEZ GALLO,JALISCO,MARIA GRACIELA ROCHA GUTIERREZ,Pay Pina,MLA,85.0,1.0,1
2,3,2029,1,1126,1048394,1309,1,6.76,0,0.0,2161 IRAPUATO GUADALUPE,GUANAJUATO,ARACELI ESTRADA PALATO,Bimbunuelos,BIM,66.0,4.0,1
3,4,2022,1,1221,2036290,1278,20,90.00,0,0.0,2152 ALTAMIRANO_BM,GUERRERO,EULOGIA,Nito,BIM,62.0,1.0,20
4,4,4041,1,1054,153108,5707,3,48.00,0,0.0,2368 TAMPICO BIMBO,TAMAULIPAS,ENCINO 2,Tostada Ondulada Tubo,MR,360.0,30.0,3


## Stats exploration (not necessary)

In [ ]:
# create specific df for stats
df_stats = df

In [10]:
# sales_unit_this_week
df_stats['sales_unit_this_week-zscore'] = stats.zscore(df_stats['sales_unit_this_week'])
df_stats[['sales_unit_this_week','sales_unit_this_week-zscore']].describe().round(3)

,sales_unit_this_week,sales_unit_this_week-zscore
count,971976.000,971976.000
mean,7.324,-0.000
std,21.453,1.000
min,0.000,-0.341
25%,2.000,-0.248
50%,3.000,-0.202
75%,7.000,-0.015
max,5000.000,232.725


In [11]:
# sales_this_week
df_stats['sales_this_week-zscore'] = stats.zscore(df_stats['sales_this_week'])
df_stats[['sales_this_week','sales_this_week-zscore']].describe().round(3)

,sales_this_week,sales_this_week-zscore
count,971976.000,971976.000
mean,68.568,0.000
std,288.706,1.000
min,0.000,-0.237
25%,16.760,-0.179
50%,30.000,-0.134
75%,56.250,-0.043
max,50353.220,174.173


In [12]:
# returns_unit_next_week
df_stats['returns_unit_next_week-zscore'] = stats.zscore(df_stats['returns_unit_next_week'])
df_stats[['returns_unit_next_week','returns_unit_next_week-zscore']].describe().round(3)

,returns_unit_next_week,returns_unit_next_week-zscore
count,971976.000,971976.000
mean,0.128,-0.000
std,2.510,1.000
min,0.000,-0.051
25%,0.000,-0.051
50%,0.000,-0.051
75%,0.000,-0.051
max,1241.000,494.383


In [13]:
# returns_next_week
df_stats['returns_next_week-zscore'] = stats.zscore(df_stats['returns_next_week'])
df_stats[['returns_next_week','returns_next_week-zscore']].describe().round(3)

,returns_next_week,returns_next_week-zscore
count,971976.000,971976.000
mean,1.297,0.000
std,53.984,1.000
min,0.000,-0.024
25%,0.000,-0.024
50%,0.000,-0.024
75%,0.000,-0.024
max,49500.000,916.914


In [14]:
# target
df_stats['target-zscore'] = stats.zscore(df_stats['target'])
df_stats[['target','target-zscore']].describe().round(3)

,target,target-zscore
count,971976.000,971976.000
mean,7.235,-0.000
std,21.254,1.000
min,0.000,-0.340
25%,2.000,-0.246
50%,3.000,-0.199
75%,6.000,-0.058
max,5000.000,234.913


## Outlier finder
important stuff starts here

\
define function to find outliers in a specific column

In [40]:
def find_outliers(col):
    from scipy import stats
    z = np.abs(stats.zscore(col))
    idx_outliers = np.where(z>1.5,True,False)
    return pd.Series(idx_outliers,index=col.index)

In [41]:
find_outliers(df['target'])

0          True
1         False
2         False
3         False
4         False
          ...  
971971    False
971972    False
971973    False
971974    False
971975    False
Length: 971976, dtype: bool

choose columns for outlier elimination

In [42]:
out_columns = df[['sales_unit_this_week','sales_this_week','returns_unit_next_week','returns_next_week','target']].columns
out_columns

Index(['sales_unit_this_week', 'sales_this_week', 'returns_unit_next_week',
       'returns_next_week', 'target'],
      dtype='object')

find outliers in all desired columns

In [43]:
df_outliers = pd.DataFrame()
for col in out_columns:
    df_outliers[col] = find_outliers(df[col])
df_outliers.head()

,sales_unit_this_week,sales_this_week,returns_unit_next_week,returns_next_week,target
0,True,False,False,False,True
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False


create variable that is true for any column with at least one outlier

In [44]:
test_out = df_outliers.apply(lambda x: np.any(x), axis=1)

In [45]:
print(len(test_out), df_outliers.shape)
test_out

971976 (971976, 5)


0          True
1         False
2         False
3         False
4         False
          ...  
971971    False
971972    False
971973    False
971974    False
971975    False
Length: 971976, dtype: bool

In [46]:
np.sum(test_out)

35312

## create final dataframe

In [47]:
df_clean = df.loc[test_out==False]
df_clean.describe()

,week_number,sales_depot_id,sales_channel_id,route_id,client_id,product_id,sales_unit_this_week,sales_this_week,returns_unit_next_week,returns_next_week,weight,target
count,936664.000000,936664.000000,936664.000000,936664.000000,9.366640e+05,936664.000000,936664.000000,936664.000000,936664.000000,936664.000000,936664.000000,936664.000000
mean,5.946232,2537.299202,1.355749,2107.157105,1.790573e+06,20634.228583,5.170975,45.553490,0.039746,0.439291,192.380398,5.138262
std,2.013173,4084.437642,1.419792,1478.590741,1.833027e+06,18650.665556,5.562449,52.120537,0.263132,3.087689,195.184467,5.574517
min,3.000000,1110.000000,1.000000,1.000000,6.500000e+01,41.000000,0.000000,0.000000,0.000000,0.000000,1.320000,0.000000
25%,4.000000,1311.000000,1.000000,1161.000000,3.531200e+05,1242.000000,2.000000,16.670000,0.000000,0.000000,80.000000,2.000000
50%,6.000000,1613.000000,1.000000,1281.000000,1.180745e+06,30532.000000,3.000000,29.490000,0.000000,0.000000,120.000000,3.000000
75%,8.000000,2034.000000,1.000000,2802.000000,2.364550e+06,37361.000000,6.000000,53.280000,0.000000,0.000000,255.000000,6.000000
max,9.000000,25759.000000,11.000000,9835.000000,1.998863e+07,49994.000000,39.000000,501.600000,3.000000,81.900000,12000.000000,39.000000


In [56]:
# NEEDS TARGET CREATION
df_clean.to_csv('data/02-zscore15-sem-target.csv',index=False)

## run same process for all numeric variables (including ids)

In [51]:
df_outliers_ids = pd.DataFrame()
for col in df.describe().columns:
    df_outliers_ids[col] = find_outliers(df[col])
df_outliers_ids.head()

,week_number,sales_depot_id,sales_channel_id,route_id,client_id,product_id,sales_unit_this_week,sales_this_week,returns_unit_next_week,returns_next_week,weight,target
0,False,False,False,False,False,False,True,False,False,False,False,True
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False


In [52]:
test_out_ids = df_outliers_ids.apply(lambda x: np.any(x), axis=1)
df_clean_ids = df.loc[test_out_ids==False]
df_clean_ids.describe()

,week_number,sales_depot_id,sales_channel_id,route_id,client_id,product_id,sales_unit_this_week,sales_this_week,returns_unit_next_week,returns_next_week,weight,target
count,554354.000000,554354.000000,554354.000000,554354.000000,5.543540e+05,554354.000000,554354.000000,554354.000000,554354.000000,554354.000000,554354.000000,554354.000000
mean,5.437581,1745.320698,1.005128,1564.855672,1.394841e+06,19668.965481,5.021340,40.321439,0.040434,0.425408,156.962911,4.988064
std,1.719238,663.958767,0.071430,607.178748,1.353621e+06,18601.464563,5.374886,43.146150,0.265256,2.933324,111.449165,5.386491
min,3.000000,1111.000000,1.000000,1.000000,7.720000e+02,72.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000
25%,4.000000,1245.000000,1.000000,1145.000000,3.166560e+05,1240.000000,2.000000,16.300000,0.000000,0.000000,80.000000,2.000000
50%,5.000000,1556.000000,1.000000,1250.000000,9.407840e+05,8921.000000,3.000000,27.180000,0.000000,0.000000,120.000000,3.000000
75%,7.000000,2021.000000,1.000000,2024.000000,2.084145e+06,37057.000000,6.000000,48.000000,0.000000,0.000000,210.000000,6.000000
max,8.000000,4066.000000,2.000000,4201.000000,4.541431e+06,48806.000000,39.000000,501.600000,3.000000,81.480000,540.000000,39.000000


In [55]:
# NEEDS TARGET CREATION
df_clean_ids.to_csv('data/02-zscore-inclui-ids-sem-target.csv',index=False)